# NDIF Monitor - Basic Trace

**Model:** `meta-llama/Llama-3.1-405B-Instruct`

This notebook tests nnsight + NDIF functionality. Generated by [NDIF Monitor](https://github.com/davidbau/ndif-monitor).

## Setup

First, configure your API keys in Colab Secrets (🔑 icon in left sidebar):
- `NDIF_API`: Your NDIF API key from [nnsight.net](https://nnsight.net)
- `HF_TOKEN`: Your HuggingFace token (for gated models)


In [ ]:
# Install dependencies
!pip install -q nnsight torch


In [ ]:
# Configure authentication from Colab secrets
import os

try:
    from google.colab import userdata
    NDIF_API = userdata.get('NDIF_API')
    HF_TOKEN = userdata.get('HF_TOKEN')
    print('Loaded secrets from Colab')
except:
    # Fallback for local testing
    NDIF_API = os.environ.get('NDIF_API')
    HF_TOKEN = os.environ.get('HF_TOKEN')
    print('Using environment variables')

# Configure nnsight
from nnsight import CONFIG
if NDIF_API:
    CONFIG.set_default_api_key(NDIF_API)
    print('NDIF API key configured')
else:
    print('Warning: NDIF_API not set - add it to Colab Secrets')

if HF_TOKEN:
    os.environ['HF_TOKEN'] = HF_TOKEN
    print('HuggingFace token configured')


In [ ]:
# Load model
from nnsight import LanguageModel
import time

MODEL_NAME = 'meta-llama/Llama-3.1-405B-Instruct'
print(f'Loading {{MODEL_NAME}}...')

start = time.time()
model = LanguageModel(MODEL_NAME, device_map='auto')
load_time = time.time() - start
print(f'Model loaded in {{load_time:.1f}}s')


## Basic Trace Test

Tests `model.trace()` functionality with hidden state extraction.


In [ ]:
# Run basic trace
prompt = 'The quick brown fox jumps over the lazy dog'
print(f"Running trace on: '{prompt}'")

start = time.time()
with model.trace(prompt, remote=True):
    hidden = model.model.layers[0].output[0].save()

trace_time = time.time() - start
print(f'Trace completed in {{trace_time:.1f}}s')
print(f'Hidden state shape: {{hidden.shape}}')


In [ ]:
# Validate results
import torch

# Verify shape is reasonable
assert len(hidden.shape) >= 2, f'Expected at least 2D tensor, got {hidden.shape}'
assert hidden.shape[-1] > 0, 'Hidden dimension should be positive'

# Check for NaN/Inf
assert not torch.isnan(hidden).any(), 'Hidden state contains NaN values'
assert not torch.isinf(hidden).any(), 'Hidden state contains Inf values'

print('Validation passed!')


In [ ]:
print('\n' + '=' * 50)
print('BASIC TRACE TEST PASSED ✓')
print('=' * 50)
